### 1. pandas 및 pickle 불러오기

In [1]:
import pandas as pd
import numpy as np
import os
import sklearn.metrics
import pickle as pickle



num_to_label pickle 에서 불러오려면 아래 셀 uncomment 후 경로 지정

num_to_label dict 직접 만들기

In [2]:
num_to_label = {0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}

In [3]:
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    preds = label_to_num(preds.values)
    labels = label_to_num(labels.values)

    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0


def label_to_num(label):
  num_label = []
  with open('../code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [63]:
def mixmaster(df1, df2, weight1, weight2):
    result = []
    df1= np.array(list(map(float, df1[1:-1].split(', '))))
    df2= np.array(list(map(float, df2[1:-1].split(', '))))
    result =np.mean([df1] * weight1 + [df2] * weight2, axis=0)
    result = result/result.sum()
    result = '[' + ', '.join(map(str, result)) + ']'
    return result
def mixmaster3(df1, df2,df3, weight1, weight2):
    result = []
    df1= np.array(list(map(float, df1[1:-1].split(', '))))
    df2= np.array(list(map(float, df2[1:-1].split(', '))))
    df3= np.array(list(map(float, df3[1:-1].split(', '))))
    result =np.mean([df1] * weight1 + [df2] * weight2 + [df3], axis=0)
    result = result/result.sum()
    result = '[' + ', '.join(map(str, result)) + ']'
    return result
def mixmaster5(df1, df2, df3, df4,df5, weight1, weight2):
    result = []
    df1= np.array(list(map(float, df1[1:-1].split(', '))))
    df2= np.array(list(map(float, df2[1:-1].split(', '))))
    df3= np.array(list(map(float, df3[1:-1].split(', '))))
    df4= np.array(list(map(float, df4[1:-1].split(', '))))
    df5= np.array(list(map(float, df5[1:-1].split(', '))))
    result =np.mean([df1] + [df2] + [df3]+ [df4]+ [df5], axis=0)
    result = result/result.sum()
    result = '[' + ', '.join(map(str, result)) + ']'
    return result


In [5]:
def change_label(x):
    x= np.array(list(map(float, x[1:-1].split(', '))))
    return num_to_label[np.argmax(x)]

In [61]:
def ensemble(folder_name, file_name1, file_name2, weight1 = 1, weight2 = 1):
    ens1 = pd.read_csv(f'{folder_name}/{file_name1}')
    ens2 = pd.read_csv(f'{folder_name}/{file_name2}')
    result = ens1.copy()
    
    result.probs=result.id.apply(lambda x: mixmaster(ens1.probs.iloc[x],ens2.probs.iloc[x],weight1,weight2))
    
    result.pred_label= result.probs.apply(lambda x: change_label(x))
    
    return result
    
def ensemble3(folder_name, file_name1, file_name2, file_name3, weight1 = 1, weight2 = 1):
    ens1 = pd.read_csv(f'{folder_name}/{file_name1}')
    ens2 = pd.read_csv(f'{folder_name}/{file_name2}')
    ens3 = pd.read_csv(f'{folder_name}/{file_name3}')
    result = ens1.copy()

    result.probs=result.id.apply(lambda x: mixmaster3(ens1.probs.iloc[x],ens2.probs.iloc[x],ens3.probs.iloc[x],weight1,weight2))

    result.pred_label= result.probs.apply(lambda x: change_label(x))

    return result
    
def ensemble5(folder_name, file_name1, file_name2, file_name3, file_name4, file_name5, weight1 = 1, weight2 = 1):
    ens1 = pd.read_csv(f'{folder_name}/{file_name1}')
    ens2 = pd.read_csv(f'{folder_name}/{file_name2}')
    ens3 = pd.read_csv(f'{folder_name}/{file_name3}')
    ens4 = pd.read_csv(f'{folder_name}/{file_name4}')
    ens5 = pd.read_csv(f'{folder_name}/{file_name5}')
    result = ens1.copy()

    result.probs=result.id.apply(lambda x: mixmaster5(ens1.probs.iloc[x],ens2.probs.iloc[x],ens3.probs.iloc[x], ens4.probs.iloc[x], ens5.probs.iloc[x],weight1,weight2))

    result.pred_label= result.probs.apply(lambda x: change_label(x))

    return result
    
    

In [15]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './save_best'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}
print(count_file)
for file_1 in range(count_file):
    for file_2 in range(file_1+1,count_file):
        
        print(file_1 , file_2)
        file_name = f'{file_list[file_1][6:-4]}_{file_list[file_2][6:-4]}'
        ens_result = ensemble(folder_name,file_list[file_1],file_list[file_2])
        ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
        ens_result= pd.concat([ens_result,correct],axis=1)
        f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))     
f1_scores




11
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
3 4
3 5
3 6
3 7
3 8
3 9
3 10
4 5
4 6
4 7
4 8
4 9
4 10
5 6
5 7
5 8
5 9
5 10
6 7
6 8
6 9
6 10
7 8
7 9
7 10
8 9
8 10
9 10


{'19_roberta-large8379_checkpoint-6000_5_sub_57_4_5_19_roberta-large_lr_checkpoint-8000_2_ub_71_21_see_14_rob_19_rob_sub_57_seo_01': 75.94332112720905,
 '14_rob_seo_02_13_rob_19_rob_sub_69_seo_01_ub_71_21_see_14_rob_19_rob_sub_57_seo_01': 75.87311759051588,
 'ub_71_21_see_14_rob_19_rob_sub_57_seo_01_19_roberta-large8379_checkpoint-6000_5_sub_57_4_5_19_roberta-large_lr_checkpoint-8000_21_roberta-large1_5_checkpoint-6000_1': 75.85987261146498,
 '19_roberta-large8379_checkpoint-6000_5_sub_57_4_ub_71_21_see_14_rob_19_rob_sub_57_seo_01': 75.85439516769988,
 '14_rob_seo_02_13_rob_19_rob_sub_69_seo_01_21_see_21_see_seo_02_13_rob_21_rob': 75.75757575757575,
 '19_roberta-large8379_checkpoint-6000_5_sub_57_4_14_rob_seo_02_13_rob_19_rob_sub_69_seo_01': 75.65935811884334,
 '21_seed_159_19_roberta-large8379_checkpoint-6000_5_sub_57_4_5_19_roberta-large_lr_checkpoint-8000_2': 75.6585211044113,
 '19_roberta-large8379_checkpoint-6000_5_sub_57_4_19_roberta-large8379_checkpoint-6000_5_sub_57_4_5_19_robe

In [13]:
folder_name = './ensemble'
file_list = os.listdir(folder_name)
count_file = len(file_list)
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')

correct = pd.read_csv('../data/dataset/test/dev_final.csv')
f1_scores={}
file_name1= 'train_13_roberta-large.csv'
file_name2 = 'train_19_roberta-large_lr.csv'

for weight1 in range(1,6):

    print(weight1)

    file_name = f'{file_name1[6:-4]}_{weight1}_{file_name2[6:-4]}_5'
    ens_result = ensemble(folder_name,file_name1,file_name2,weight1,5)
    ens_result.to_csv(f'./ensemble_result/{file_name}.csv')
    ens_result= pd.concat([ens_result,correct],axis=1)
    f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

for weight2 in range(1,6):

    print(weight2)

    file_name = f'{file_name1[6:-4]}_5_{file_name2[6:-4]}_{weight2}'
    ens_result = ensemble(folder_name,file_name1,file_name2,5,weight2)
    ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
    ens_result= pd.concat([ens_result,correct],axis=1)
    f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))
f1_scores


1
2
3
4
5
1
2
3
4
5


{'13_roberta-large_5_19_roberta-large_lr_5': 75.53225293930727,
 '13_roberta-large_4_19_roberta-large_lr_5': 75.38950715421304,
 '13_roberta-large_5_19_roberta-large_lr_4': 75.25789557213142,
 '13_roberta-large_3_19_roberta-large_lr_5': 75.12319186138929,
 '13_roberta-large_5_19_roberta-large_lr_3': 75.0753609392353,
 '13_roberta-large_2_19_roberta-large_lr_5': 74.97222663069354,
 '13_roberta-large_5_19_roberta-large_lr_2': 74.75067278771569,
 '13_roberta-large_1_19_roberta-large_lr_5': 74.67089611419509,
 '13_roberta-large_5_19_roberta-large_lr_1': 74.22517394054397}

In [80]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}
print(count_file)
for file_1 in range(count_file):
    for file_2 in range(file_1+1,count_file):
        print(file_1,file_2)

        for weight1 in range(1,6):

            file_name = f'{file_list[file_1][6:-4]}_{weight1}_{file_list[file_2][6:-4]}_5'
            ens_result = ensemble(folder_name,file_list[file_1],file_list[file_2],weight1,5)
            ens_result.to_csv(f'./ensemble_result/{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

        for weight2 in range(1,6):


            file_name = f'{file_list[file_1][6:-4]}_5_{file_list[file_2][6:-4]}_{weight2}'
            ens_result = ensemble(folder_name,file_list[file_1],file_list[file_2],5,weight2)
            ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)
                

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))     
f1_scores




5
0 1
1
2
3
4
5
1
2
3
4
5
0 2
1
2
3
4
5
1
2
3
4
5
0 3
1
2
3
4
5
1
2
3
4
5
0 4
1
2
3
4
5
1
2
3
4
5
1 2
1
2
3
4
5
1
2
3
4
5
1 3
1
2
3
4
5
1
2
3
4
5
1 4
1
2
3
4
5
1
2
3
4
5
2 3
1
2
3
4
5
1
2
3
4
5
2 4
1
2
3
4
5
1
2
3
4
5
3 4
1
2
3
4
5
1
2
3
4
5


{'21_roberta-large_5_19_roberta-large_lr_5': 75.45310015898251,
 '21_roberta-large_5_19_roberta-large_lr_4': 75.44891148895597,
 '21_roberta-large_4_19_roberta-large_lr_5': 75.34573199809252,
 '19_roberta-large_lr_5_sub_57_5': 75.33206831119543,
 '21_roberta-large_5_sub_57_4': 75.279307631786,
 '21_roberta-large_5_sub_57_3': 75.24034672970843,
 '21_roberta-large_5_19_roberta-large_lr_3': 75.22208121827411,
 '19_roberta-large_lr_5_sub_57_4': 75.20961873121341,
 '14_roberta-large_4_19_roberta-large_lr_5': 75.18002880460874,
 '21_roberta-large_5_sub_57_5': 75.13394264103371,
 '15_roberta-large_5_sub_57_5': 75.11992324912055,
 '19_roberta-large_lr_4_sub_57_5': 75.1064165221504,
 '15_roberta-large_4_21_roberta-large_5': 75.08783136378155,
 '15_roberta-large_4_sub_57_5': 75.08749602290806,
 '19_roberta-large_lr_5_sub_57_3': 75.07926442612556,
 '14_roberta-large_5_19_roberta-large_lr_4': 75.06410256410257,
 '14_roberta-large_5_19_roberta-large_lr_5': 75.06014434643144,
 '14_roberta-large_3_19

In [33]:
num_to_label.values()

dict_values(['no_relation', 'org:top_members/employees', 'org:members', 'org:product', 'per:title', 'org:alternate_names', 'per:employee_of', 'org:place_of_headquarters', 'per:product', 'org:number_of_employees/members', 'per:children', 'per:place_of_residence', 'per:alternate_names', 'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings', 'per:spouse', 'org:founded', 'org:political/religious_affiliation', 'org:member_of', 'per:parents', 'org:dissolved', 'per:schools_attended', 'per:date_of_death', 'per:date_of_birth', 'per:place_of_birth', 'per:place_of_death', 'org:founded_by', 'per:religion'])

In [9]:
f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))

In [27]:
file_name ='train_'+'19_roberta-large8379_checkpoint-6000_5_sub_57_4_5_19_roberta-large_lr_checkpoint-8000_21_roberta-large1_5_checkpoint-6000_1'+'.csv'
x= pd.read_csv(f'./ensemble_result/{file_name}')
x.to_csv(f'./save_best/{file_name}')
x.to_csv(f'./ensemble/{file_name}')

In [17]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_soft'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}
print(count_file)
for file_1 in range(count_file):
    for file_2 in range(file_1+1,count_file):
        
        print(file_1 , file_2)
        file_name = f'{file_list[file_1][6:-4]}_{file_list[file_2][6:-4]}'
        ens_result = ensemble(folder_name,file_list[file_1],file_list[file_2])
        ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
        ens_result= pd.concat([ens_result,correct],axis=1)
        f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))     
f1_scores




7
0 1
0 2
0 3
0 4
0 5
0 6
1 2
1 3
1 4
1 5
1 6
2 3
2 4
2 5
2 6
3 4
3 5
3 6
4 5
4 6
5 6


{'seo_02_sub_69': 75.7263057628195,
 '19_roberta-large_lr_seo_01': 75.60936753226063,
 '21_seed_159_seo_02': 75.57251908396945,
 'seo_02_19_roberta-large_lr': 75.54218774734844,
 '13_roberta-large_19_roberta-large_lr': 75.53225293930727,
 '21_seed_159_14_roberta-large': 75.49350024073182,
 '21_seed_159_sub_69': 75.42966263526417,
 '14_roberta-large_seo_02': 75.38091419406577,
 '21_seed_159_13_roberta-large': 75.25131641933939,
 '21_seed_159_19_roberta-large_lr': 75.2465797009227,
 'seo_02_seo_01': 75.19847570657352,
 '13_roberta-large_seo_01': 75.17098775250518,
 '14_roberta-large_seo_01': 75.13669990350596,
 'seo_02_13_roberta-large': 75.12256840107544,
 '21_seed_159_seo_01': 75.09163346613545,
 '14_roberta-large_19_roberta-large_lr': 75.06014434643144,
 'sub_69_seo_01': 75.02386255170221,
 '14_roberta-large_sub_69': 74.95976826520759,
 '19_roberta-large_lr_sub_69': 74.83296213808464,
 '13_roberta-large_sub_69': 74.70148065594651,
 '14_roberta-large_13_roberta-large': 74.6082507195395

In [29]:
list(f1_scores.keys())

['seo_02_sub_69',
 '19_roberta-large_lr_seo_01',
 '21_seed_159_seo_02',
 'seo_02_19_roberta-large_lr',
 '13_roberta-large_19_roberta-large_lr',
 '21_seed_159_14_roberta-large',
 '21_seed_159_sub_69',
 '14_roberta-large_seo_02',
 '21_seed_159_13_roberta-large',
 '21_seed_159_19_roberta-large_lr',
 'seo_02_seo_01',
 '13_roberta-large_seo_01',
 '14_roberta-large_seo_01',
 'seo_02_13_roberta-large',
 '21_seed_159_seo_01',
 '14_roberta-large_19_roberta-large_lr',
 'sub_69_seo_01',
 '14_roberta-large_sub_69',
 '19_roberta-large_lr_sub_69',
 '13_roberta-large_sub_69',
 '14_roberta-large_13_roberta-large']

In [38]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores.keys())

f1_scores2={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        
        print(file_1,file_2)
        name1= f'train_{keys[file_1]}.csv'
        name2= f'train_{keys[file_2]}.csv'
        file_name = f'{name1[6:-4]}_{name2[6:-4]}'
        print(name1, name2)
        ens_result = ensemble(folder_name,name1,name2)
        ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
        ens_result= pd.concat([ens_result,correct],axis=1)
        f1_scores2[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores2=dict(sorted(f1_scores2.items(), key=lambda x:-x[1]))     
f1_scores2




137
train_seo_02_sub_69.csv train_19_roberta-large_lr_seo_01.csv
train_seo_02_sub_69.csv train_21_seed_159_seo_02.csv
train_seo_02_sub_69.csv train_seo_02_19_roberta-large_lr.csv
train_seo_02_sub_69.csv train_13_roberta-large_19_roberta-large_lr.csv
train_seo_02_sub_69.csv train_21_seed_159_14_roberta-large.csv
train_19_roberta-large_lr_seo_01.csv train_21_seed_159_seo_02.csv
train_19_roberta-large_lr_seo_01.csv train_seo_02_19_roberta-large_lr.csv
train_19_roberta-large_lr_seo_01.csv train_13_roberta-large_19_roberta-large_lr.csv
train_19_roberta-large_lr_seo_01.csv train_21_seed_159_14_roberta-large.csv
train_21_seed_159_seo_02.csv train_seo_02_19_roberta-large_lr.csv
train_21_seed_159_seo_02.csv train_13_roberta-large_19_roberta-large_lr.csv
train_21_seed_159_seo_02.csv train_21_seed_159_14_roberta-large.csv
train_seo_02_19_roberta-large_lr.csv train_13_roberta-large_19_roberta-large_lr.csv
train_seo_02_19_roberta-large_lr.csv train_21_seed_159_14_roberta-large.csv
train_13_roberta-

{'seo_02_sub_69_21_seed_159_14_roberta-large': 76.18132308185169,
 'seo_02_sub_69_seo_02_19_roberta-large_lr': 76.0375258387661,
 'seo_02_sub_69_21_seed_159_seo_02': 76.02227525855211,
 '21_seed_159_seo_02_seo_02_19_roberta-large_lr': 75.97516319057475,
 'seo_02_sub_69_13_roberta-large_19_roberta-large_lr': 75.93685217668634,
 'seo_02_19_roberta-large_lr_13_roberta-large_19_roberta-large_lr': 75.86866432897675,
 '19_roberta-large_lr_seo_01_21_seed_159_seo_02': 75.86096938775509,
 'seo_02_sub_69_19_roberta-large_lr_seo_01': 75.85218222363811,
 'seo_02_19_roberta-large_lr_21_seed_159_14_roberta-large': 75.84548805898382,
 '21_seed_159_seo_02_21_seed_159_14_roberta-large': 75.81365666879387,
 '19_roberta-large_lr_seo_01_13_roberta-large_19_roberta-large_lr': 75.7851107922844,
 '21_seed_159_seo_02_13_roberta-large_19_roberta-large_lr': 75.77303155881414,
 '19_roberta-large_lr_seo_01_seo_02_19_roberta-large_lr': 75.7503573130062,
 '19_roberta-large_lr_seo_01_21_seed_159_14_roberta-large': 7

In [39]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores2.keys())

f1_scores3={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        
        print(file_1,file_2)
        name1= f'train_{keys[file_1]}.csv'
        name2= f'train_{keys[file_2]}.csv'
        file_name = f'{name1[6:-4]}_{name2[6:-4]}'
        print(name1, name2)
        ens_result = ensemble(folder_name,name1,name2)
        ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
        ens_result= pd.concat([ens_result,correct],axis=1)
        f1_scores3[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores3=dict(sorted(f1_scores3.items(), key=lambda x:-x[1]))     
f1_scores3




147
0 1
train_seo_02_sub_69_21_seed_159_14_roberta-large.csv train_seo_02_sub_69_seo_02_19_roberta-large_lr.csv
0 2
train_seo_02_sub_69_21_seed_159_14_roberta-large.csv train_seo_02_sub_69_21_seed_159_seo_02.csv
0 3
train_seo_02_sub_69_21_seed_159_14_roberta-large.csv train_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv
0 4
train_seo_02_sub_69_21_seed_159_14_roberta-large.csv train_seo_02_sub_69_13_roberta-large_19_roberta-large_lr.csv
0 5
train_seo_02_sub_69_21_seed_159_14_roberta-large.csv train_seo_02_19_roberta-large_lr_13_roberta-large_19_roberta-large_lr.csv
1 2
train_seo_02_sub_69_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_21_seed_159_seo_02.csv
1 3
train_seo_02_sub_69_seo_02_19_roberta-large_lr.csv train_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv
1 4
train_seo_02_sub_69_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_13_roberta-large_19_roberta-large_lr.csv
1 5
train_seo_02_sub_69_seo_02_19_roberta-large_lr.csv train_seo_02_19_roberta-large_lr_13_roberta-la

{'seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr': 76.48562300319489,
 'seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_21_seed_159_seo_02': 76.34391449992025,
 'seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_seo_02_19_roberta-large_lr': 76.20722737448033,
 'seo_02_sub_69_seo_02_19_roberta-large_lr_21_seed_159_seo_02_seo_02_19_roberta-large_lr': 76.1783439490446,
 'seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_13_roberta-large_19_roberta-large_lr': 76.14180772916,
 'seo_02_sub_69_21_seed_159_seo_02_21_seed_159_seo_02_seo_02_19_roberta-large_lr': 76.12225405921681,
 'seo_02_sub_69_21_seed_159_seo_02_seo_02_sub_69_13_roberta-large_19_roberta-large_lr': 76.09320140440471,
 'seo_02_sub_69_21_seed_159_seo_02_seo_02_19_roberta-large_lr_13_roberta-large_19_roberta-large_lr': 76.09180745935609,
 '21_seed_159_seo_02_seo_02_19_roberta-large_lr_seo_02_sub_69_13_roberta-large_19_roberta-large_lr': 76.09180745935609,
 'seo_02_s

In [40]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores3.keys())

f1_scores4={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        
        print(file_1,file_2)
        name1= f'train_{keys[file_1]}.csv'
        name2= f'train_{keys[file_2]}.csv'
        file_name = f'{name1[6:-4]}_{name2[6:-4]}'
        print(name1, name2)
        ens_result = ensemble(folder_name,name1,name2)
        ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
        ens_result= pd.concat([ens_result,correct],axis=1)
        f1_scores4[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores4=dict(sorted(f1_scores4.items(), key=lambda x:-x[1]))     
f1_scores4




162
0 1
train_seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_21_seed_159_seo_02.csv
0 2
train_seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_seo_02_19_roberta-large_lr.csv
0 3
train_seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_seo_02_19_roberta-large_lr_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv
0 4
train_seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_13_roberta-large_19_roberta-large_lr.csv
0 5
train_seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv train_seo_02_sub_69_21_seed_159_seo_02_21_seed_159_seo_02_seo_02_19_roberta-large_lr.csv
1 2
train_s

{'seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_seo_02_19_roberta-large_lr': 76.46871008939975,
 'seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_21_seed_159_seo_02_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_13_roberta-large_19_roberta-large_lr': 76.45836662937509,
 'seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr_seo_02_sub_69_21_seed_159_seo_02_21_seed_159_seo_02_seo_02_19_roberta-large_lr': 76.42276422764228,
 'seo_02_sub_69_21_seed_159_14_roberta-large_21_seed_159_seo_02_seo_02_19_roberta-large_lr_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_21_seed_159_seo_02': 76.4030612244898,
 'seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_21_seed_159_seo_02_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_sub_69_seo_02_19_roberta-large_lr': 76.37002716088833,
 'seo_02_sub_69_21_seed_159_14_roberta-large_21_se

In [47]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_soft'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}
print(count_file)
for file_1 in range(count_file):
    for file_2 in range(file_1+1,count_file):
        for file_3 in range(file_2+1, count_file):
        
            print(file_1 , file_2, file_3)
            file_name = f'{file_list[file_1][6:-4]}_{file_list[file_2][6:-4]}_{file_list[file_3][6:-4]}'
            ens_result = ensemble3(folder_name,file_list[file_1],file_list[file_2],file_list[file_3])
            ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))     
f1_scores




7
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 2 2
0 2 3
0 2 4
0 2 5
0 2 6
0 3 3
0 3 4
0 3 5
0 3 6
0 4 4
0 4 5
0 4 6
0 5 5
0 5 6
0 6 6
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 3 3
1 3 4
1 3 5
1 3 6
1 4 4
1 4 5
1 4 6
1 5 5
1 5 6
1 6 6
2 3 3
2 3 4
2 3 5
2 3 6
2 4 4
2 4 5
2 4 6
2 5 5
2 5 6
2 6 6
3 4 4
3 4 5
3 4 6
3 5 5
3 5 6
3 6 6
4 5 5
4 5 6
4 6 6
5 6 6


{'21_seed_159_seo_02_sub_69': 76.2436224489796,
 '21_seed_159_14_roberta-large_seo_02': 76.06988299406956,
 '21_seed_159_seo_02_13_roberta-large': 76.00191601468946,
 'seo_02_19_roberta-large_lr_sub_69': 75.96627962462223,
 'seo_02_13_roberta-large_19_roberta-large_lr': 75.88291441298122,
 '21_seed_159_seo_02_19_roberta-large_lr': 75.84556477345245,
 '21_seed_159_seo_02_seo_02': 75.8390329250835,
 '21_seed_159_13_roberta-large_sub_69': 75.79384075315141,
 'seo_02_19_roberta-large_lr_seo_01': 75.78846766486141,
 '21_seed_159_13_roberta-large_19_roberta-large_lr': 75.77975811584979,
 '14_roberta-large_seo_02_seo_02': 75.772047118752,
 '13_roberta-large_19_roberta-large_lr_seo_01': 75.74210022342803,
 'seo_02_13_roberta-large_seo_01': 75.72722937529804,
 '14_roberta-large_seo_02_sub_69': 75.71634384504561,
 '14_roberta-large_seo_02_19_roberta-large_lr': 75.7085668534828,
 '21_seed_159_seo_02_seo_01': 75.67826364506863,
 'seo_02_13_roberta-large_sub_69': 75.66964285714285,
 '14_roberta-lar

In [48]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores.keys())

f1_scores2={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        for file_3 in range(file_2+1,6):
        
            print(file_1,file_2,file_3)
            name1= f'train_{keys[file_1]}.csv'
            name2= f'train_{keys[file_2]}.csv'
            name3= f'train_{keys[file_3]}.csv'
            file_name = f'{name1[6:-4]}_{name2[6:-4]}_{name3[6:-4]}'
            ens_result = ensemble3(folder_name,name1,name2,name3)
            ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores2[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores2=dict(sorted(f1_scores2.items(), key=lambda x:-x[1]))     
f1_scores2




275
0 1 2
0 1 3
0 1 4
0 1 5
0 2 3
0 2 4
0 2 5
0 3 4
0 3 5
0 4 5
1 2 3
1 2 4
1 2 5
1 3 4
1 3 5
1 4 5
2 3 4
2 3 5
2 4 5
3 4 5


{'21_seed_159_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_seo_02_13_roberta-large_19_roberta-large_lr': 76.38223074464686,
 '21_seed_159_14_roberta-large_seo_02_21_seed_159_seo_02_13_roberta-large_seo_02_19_roberta-large_lr_sub_69': 76.38223074464686,
 '21_seed_159_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_21_seed_159_seo_02_13_roberta-large': 76.32294164668265,
 '21_seed_159_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_seo_02_19_roberta-large_lr_sub_69': 76.31956625737521,
 '21_seed_159_seo_02_sub_69_21_seed_159_14_roberta-large_seo_02_21_seed_159_seo_02_19_roberta-large_lr': 76.23461722870385,
 '21_seed_159_14_roberta-large_seo_02_21_seed_159_seo_02_13_roberta-large_21_seed_159_seo_02_19_roberta-large_lr': 76.23160588611645,
 '21_seed_159_seo_02_sub_69_seo_02_13_roberta-large_19_roberta-large_lr_21_seed_159_seo_02_19_roberta-large_lr': 76.19655392469687,
 '21_seed_159_seo_02_13_roberta-large_seo_02_19_roberta-large_lr_sub_69_21_seed_159_seo_02_19_roberta-large_lr':

In [51]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores2.keys())

f1_scores3={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        for file_3 in range(file_2+1,6):
        
            print(file_1,file_2,file_3)
            name1= f'train_{keys[file_1]}.csv'
            name2= f'train_{keys[file_2]}.csv'
            name3= f'train_{keys[file_3]}.csv'
            file_name = f'{name1[6:-4]}_{name2[6:-4]}_{name3[6:-4]}'[::2]
            ens_result = ensemble3(folder_name,name1,name2,name3)
            ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores3[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores3=dict(sorted(f1_scores3.items(), key=lambda x:-x[1]))     
f1_scores3




296
0 1 2
0 1 3
0 1 4
0 1 5
0 2 3
0 2 4
0 2 5
0 3 4
0 3 5
0 4 5
1 2 3
1 2 4
1 2 5
1 3 4
1 3 5
1 4 5
2 3 4
2 3 5
2 4 5
3 4 5


{'2_ed19so0_u_92_ed191_oet-ag_e_2so0_3rbralre1_oet-ag_r2_ed191_oet-ag_e_22_ed19so0_3rbralreso0_9rbralrel_u_92_ed19so0_u_92_ed191_oet-ag_e_2so0_9rbralrel_u_9': 76.55425922966276,
 '2_ed19so0_u_92_ed191_oet-ag_e_2so0_3rbralre1_oet-ag_r2_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_3rbralre2_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_9rbralrel': 76.49784310592747,
 '2_ed191_oet-ag_e_22_ed19so0_3rbralreso0_9rbralrel_u_92_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_3rbralre2_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_9rbralrel': 76.49784310592747,
 '2_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_3rbralre2_ed19so0_u_92_ed191_oet-ag_e_2so0_9rbralrel_u_92_ed191_oet-ag_e_22_ed19so0_3rbralre2_ed19so0_9rbralrel': 76.49784310592747,
 '2_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_3rbralre2_ed19so0_u_92_ed191_oet-ag_e_2so0_9rbralrel_u_92_ed19so0_u_92_ed191_oet-ag_e_22_ed19so0_9rbralrel': 76.43678160919542,
 '2_ed19so0_u_92_ed191_oet-ag_e_2so0_3rbralre1_oet-ag_r2_ed19so0_u_92_ed191_oet-ag_e_2so0_9rbralrel_u_92_ed19so0_u_92_ed191

In [52]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores3.keys())

f1_scores4={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        for file_3 in range(file_2+1,6):
        
            print(file_1,file_2,file_3)
            name1= f'train_{keys[file_1]}.csv'
            name2= f'train_{keys[file_2]}.csv'
            name3= f'train_{keys[file_3]}.csv'
            file_name = f'{name1[6:-4]}_{name2[6:-4]}_{name3[6:-4]}'[::3]
            ens_result = ensemble3(folder_name,name1,name2,name3)
            ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores4[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores4=dict(sorted(f1_scores4.items(), key=lambda x:-x[1]))     
f1_scores4




316
0 1 2
0 1 3
0 1 4
0 1 5
0 2 3
0 2 4
0 2 5
0 3 4
0 3 5
0 4 5
1 2 3
1 2 4
1 2 5
1 3 4
1 3 5
1 4 5
2 3 4
2 3 5
2 4 5
3 4 5


{'2ds_9e9o-_20raeo-__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2d1eae2ds_bls_bll__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2ds_9e9o-_2e90raee90__1_tg_o9rr_9e9o-_2e90raee90rae': 76.49784310592747,
 '2ds_9e9o-_20raeo-__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2ds_9e9o-_2e90raee90__1_tg_o9rr_9e90__1_tg__1o9rr_e90__1_tg_o3rr_tg2ds_9e9o-_20raeu2ds_9e9o-_2e90rae': 76.49784310592747,
 '2d1eae2ds_bls_bll__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2ds_9e9o-_2e90raee90__1_tg_o9rr_9e90__1_tg__1o9rr_e90__1_tg_o3rr_tg2ds_9e9o-_20raeu2ds_9e9o-_2e90rae': 76.49784310592747,
 '2ds_9e9o-_2e90raee90__1_tg_o9rr_9e9o-_2e90raee90rae2ds_9e9o-_2e90raee90__1_tg_o9rr_9e90__1_tg__1o9rr_e90__1_tg_o3rr_tg2ds_9e9o-_20raeu2ds_9e9o-_2e90rae': 76.49784310592747,
 '2ds_9e9o-_20raeo-__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2d1eae2ds_bls_bll__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2ds_9e9o-_20raeo-__1ou2d1eaes_bll__1ou2d1eae2ds_bll': 76.45836662937509,
 '2ds_9e9o-_20raeo-__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2ds_9e9o-_2e90raee90__1_tg_o9rr_9e9o-_2e90raee90rae2ds_9e9o-_20raeo-

In [64]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_soft'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}
print(count_file)
for file_1 in range(count_file):
    for file_2 in range(file_1+1,count_file):
        for file_3 in range(file_2+1, count_file):
            for file_4 in range(file_3+1, count_file):
                for file_5 in range(file_4+1, count_file):
        
                    print(file_1 , file_2, file_3,file_4, file_5)
                    file_name = f'{file_list[file_1][6:-4]}_{file_list[file_2][6:-4]}_{file_list[file_3][6:-4]}_{file_list[file_4][6:-4]}_{file_list[file_5][6:-4]}'[::2]
                    ens_result = ensemble5(folder_name,file_list[file_1],file_list[file_2],file_list[file_3],file_list[file_4],file_list[file_5])
                    ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
                    ens_result= pd.concat([ens_result,correct],axis=1)
                    f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))     
f1_scores




8
0 1 2 3 4
0 1 2 3 5
0 1 2 3 6
0 1 2 3 7
0 1 2 4 5
0 1 2 4 6
0 1 2 4 7
0 1 2 5 6
0 1 2 5 7
0 1 2 6 7
0 1 3 4 5
0 1 3 4 6
0 1 3 4 7
0 1 3 5 6
0 1 3 5 7
0 1 3 6 7
0 1 4 5 6
0 1 4 5 7
0 1 4 6 7
0 1 5 6 7
0 2 3 4 5
0 2 3 4 6
0 2 3 4 7
0 2 3 5 6
0 2 3 5 7
0 2 3 6 7
0 2 4 5 6
0 2 4 5 7
0 2 4 6 7
0 2 5 6 7
0 3 4 5 6
0 3 4 5 7
0 3 4 6 7
0 3 5 6 7
0 4 5 6 7
1 2 3 4 5
1 2 3 4 6
1 2 3 4 7
1 2 3 5 6
1 2 3 5 7
1 2 3 6 7
1 2 4 5 6
1 2 4 5 7
1 2 4 6 7
1 2 5 6 7
1 3 4 5 6
1 3 4 5 7
1 3 4 6 7
1 3 5 6 7
1 4 5 6 7
2 3 4 5 6
2 3 4 5 7
2 3 4 6 7
2 3 5 6 7
2 4 5 6 7
3 4 5 6 7


{'2_ed19so0_5_3rbralresb6': 76.46306363927087,
 '2_ed191_oet-ag_e_265sb6': 76.25760974046779,
 '2_ed191_oet-ag_e_2sb6_e_1': 76.13399583266548,
 'so0_5_3rbralre1_oet-ag_rso0': 76.12655800575264,
 '2_ed19so0_3rbralre1_oet-ag_rso0': 76.09320140440471,
 '2_ed191_oet-ag_e_265so0': 76.08207758897082,
 '2_ed191_oet-ag_5_u_9so0': 76.08207758897082,
 '1_oet-ag_e_265sb6_e_1': 76.0806916426513,
 '2_ed19so0_5_9rbralrel_u_9': 76.06442353691595,
 '2_ed19so0_5_u_9so0': 76.05813767768726,
 '2_ed191_oet-ag_e_2651_oet-ag_r': 76.05769230769232,
 '2_ed19so0_5_9rbralrel_e_1': 76.05678736640613,
 '2_ed191_oet-ag_e_21_oet-ag_rsb6': 76.04099935938501,
 'so0_3rbralre1_oet-ag_rsb6_e_1': 76.0306807286673,
 'so0_5_3rbralresb6_e_1': 76.02805227924769,
 '2_ed19so0_5_3rbralreso0': 76.02171136653895,
 '2_ed191_oet-ag_e_21_oet-ag_rso0': 75.998075998076,
 '1_oet-ag_e_2651_oet-ag_rso0': 75.99358974358975,
 'so0_5_9rbralrel_u_9so0': 75.99298133673632,
 '1_oet-ag_5_9rbralrel_u_9so0': 75.98715890850724,
 '2_ed191_oet-ag_e_

In [66]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores.keys())

f1_scores2={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        for file_3 in range(file_2+1,6):
        
            print(file_1,file_2,file_3)
            name1= f'train_{keys[file_1]}.csv'
            name2= f'train_{keys[file_2]}.csv'
            name3= f'train_{keys[file_3]}.csv'
            file_name = f'{name1[6:-4]}_{name2[6:-4]}_{name3[6:-4]}'
            ens_result = ensemble3(folder_name,name1,name2,name3)
            ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
            ens_result= pd.concat([ens_result,correct],axis=1)
            f1_scores2[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores2=dict(sorted(f1_scores2.items(), key=lambda x:-x[1]))     
f1_scores2




397
0 1 2
0 1 3
0 1 4
0 1 5
0 2 3
0 2 4
0 2 5
0 3 4
0 3 5
0 4 5
1 2 3
1 2 4
1 2 5
1 3 4
1 3 5
1 4 5
2 3 4
2 3 5
2 4 5
3 4 5


{'2_ed19so0_5_3rbralresb6_2_ed191_oet-ag_e_265sb6_so0_5_3rbralre1_oet-ag_rso0': 76.36712503997441,
 '2_ed191_oet-ag_e_265sb6_so0_5_3rbralre1_oet-ag_rso0_2_ed191_oet-ag_e_265so0': 76.36247402908741,
 '2_ed19so0_5_3rbralresb6_so0_5_3rbralre1_oet-ag_rso0_2_ed191_oet-ag_e_265so0': 76.29854562889562,
 '2_ed19so0_5_3rbralresb6_2_ed191_oet-ag_e_2sb6_e_1_so0_5_3rbralre1_oet-ag_rso0': 76.29096722621902,
 '2_ed191_oet-ag_e_2sb6_e_1_2_ed19so0_3rbralre1_oet-ag_rso0_2_ed191_oet-ag_e_265so0': 76.27579587266037,
 '2_ed19so0_5_3rbralresb6_2_ed191_oet-ag_e_265sb6_2_ed191_oet-ag_e_2sb6_e_1': 76.24539484222329,
 '2_ed191_oet-ag_e_265sb6_2_ed19so0_3rbralre1_oet-ag_rso0_2_ed191_oet-ag_e_265so0': 76.23160588611645,
 '2_ed191_oet-ag_e_265sb6_2_ed191_oet-ag_e_2sb6_e_1_2_ed191_oet-ag_e_265so0': 76.21032382173773,
 '2_ed19so0_5_3rbralresb6_2_ed191_oet-ag_e_265sb6_2_ed191_oet-ag_e_265so0': 76.1996161228407,
 '2_ed191_oet-ag_e_265sb6_2_ed191_oet-ag_e_2sb6_e_1_so0_5_3rbralre1_oet-ag_rso0': 76.1874300335839,
 '2_ed

In [ ]:
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_result'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

keys = list(f1_scores2.keys())

f1_scores3={}
print(count_file)
for file_1 in range(6):
    for file_2 in range(file_1+1,6):
        for file_3 in range(file_2+1,6):
          for file_4 in range(file_3+1,6):
            for file_5 in range(file_4+1,6):

        
              print(file_1,file_2,file_3,file_4,file_5)
              name1= f'train_{keys[file_1]}.csv'
              name2= f'train_{keys[file_2]}.csv'
              name3= f'train_{keys[file_3]}.csv'
              name4= f'train_{keys[file_4]}.csv'
              name5= f'train_{keys[file_5]}.csv'
              file_name = f'{file_list[file_1][6:-4]}_{file_list[file_2][6:-4]}_{file_list[file_3][6:-4]}_{file_list[file_4][6:-4]}_{file_list[file_5][6:-4]}'[::3]

              ens_result = ensemble5(folder_name,name1,name2,name3, name4,name5)
              ens_result.to_csv(f'./ensemble_result/train_{file_name}.csv')
              ens_result= pd.concat([ens_result,correct],axis=1)
              f1_scores2[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores2=dict(sorted(f1_scores2.items(), key=lambda x:-x[1]))     
f1_scores2




In [67]:
name ='2_ed19so0_u_92_ed191_oet-ag_e_2so0_3rbralre1_oet-ag_r2_ed191_oet-ag_e_22_ed19so0_3rbralreso0_9rbralrel_u_92_ed19so0_u_92_ed191_oet-ag_e_2so0_9rbralrel_u_9'
bestmodel=pd.read_csv(f'./ensemble_result/train_{name}.csv')
bestmodel.to_csv('./ensemble_soft/final7655.csv')
len(bestmodel)

7765

In [56]:
f1_scores4['2ds_9e9o-_20raeo-__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2d1eae2ds_bls_bll__1ou2d1eae2ds_bl2ds_9e9o-_2e90rae2ds_9e9o-_2e90raee90__1_tg_o9rr_9e9o-_2e90raee90rae']

76.49784310592747